In [1]:
import pandas as pd
from sklearn.linear_model import Ridge, BayesianRidge, ElasticNet, RidgeCV, ElasticNetCV
from sklearn.model_selection import cross_val_score, ShuffleSplit
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import numpy as np


In [5]:
#store skater stats csv into dataframe 
game_skater_df = pd.read_csv("data/game_skater_stats.csv")
game_skater_df.head(10)

,game_id,player_id,team_id,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,...,faceoffTaken,takeaways,giveaways,shortHandedGoals,shortHandedAssists,blocked,plusMinus,evenTimeOnIce,shortHandedTimeOnIce,powerPlayTimeOnIce
0,2016020045,8468513,4,955,1,0,0,2.0,0,0,...,0,1.0,1.0,0,0,1.0,1,858,97,0
1,2016020045,8476906,4,1396,1,0,4,2.0,0,0,...,0,1.0,2.0,0,0,2.0,0,1177,0,219
2,2016020045,8474668,4,915,0,0,1,1.0,0,0,...,0,2.0,0.0,0,0,0.0,-1,805,0,110
3,2016020045,8473512,4,1367,3,0,0,0.0,0,2,...,27,0.0,0.0,0,0,0.0,-1,1083,19,265
4,2016020045,8471762,4,676,0,0,3,2.0,0,0,...,0,0.0,1.0,0,0,0.0,-1,613,63,0
5,2016020045,8478439,4,1008,1,0,2,0.0,0,0,...,0,1.0,0.0,0,0,0.0,-1,911,0,97
6,2016020045,8479648,4,630,0,0,0,2.0,0,0,...,0,0.0,0.0,0,0,0.0,-1,630,0,0
7,2016020045,8470159,4,409,0,0,0,0.0,0,0,...,7,1.0,0.0,0,0,0.0,-1,360,49,0
8,2016020045,8478500,4,1275,0,0,0,1.0,0,0,...,0,0.0,1.0,0,0,0.0,-5,1015,139,121
9,2016020045,8475430,4,1029,0,0,1,5.0,0,0,...,0,0.0,1.0,0,0,1.0,1,988,41,0


In [6]:
#create new dataframe with select columms 
new_game_skater_df = game_skater_df[['player_id', 'timeOnIce', 'assists', 'goals', 'shots', 'blocked', 'powerPlayGoals', 'powerPlayAssists', 'shortHandedGoals', 'shortHandedAssists', ]].copy()
new_game_skater_df .head()

,player_id,timeOnIce,assists,goals,shots,blocked,powerPlayGoals,powerPlayAssists,shortHandedGoals,shortHandedAssists
0,8468513,955,1,0,0,1.0,0,0,0,0
1,8476906,1396,1,0,4,2.0,0,0,0,0
2,8474668,915,0,0,1,0.0,0,0,0,0
3,8473512,1367,3,0,0,0.0,0,2,0,0
4,8471762,676,0,0,3,0.0,0,0,0,0


In [7]:
#goalie stats csv into dataframe
goalie_stats_df= pd.read_csv('data/game_goalie_stats.csv')
goalie_stats_df.head()

,game_id,player_id,team_id,timeOnIce,assists,goals,pim,shots,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decision,savePercentage,powerPlaySavePercentage,evenStrengthSavePercentage
0,2016020045,8473607,4,1504,0,0,0,16,12,1,0,11,0,13,3,NaN,75.000000,33.333333,84.615385
1,2016020045,8473461,4,2011,0,0,0,11,9,1,0,8,0,10,1,L,81.818182,100.000000,80.000000
2,2016020045,8470645,16,3598,0,0,0,27,23,2,0,21,0,23,4,W,85.185185,50.000000,91.304348
3,2017020812,8468011,24,3696,0,0,0,33,30,1,2,27,3,28,2,W,90.909091,50.000000,96.428571
4,2017020812,8475215,7,3539,0,0,0,33,29,4,1,24,1,27,5,L,87.878788,80.000000,88.888889


In [8]:
#store player info csv into dataframe 
player_info_df = pd.read_csv('data/player_info.csv')
player_info_df.head()

,player_id,firstName,lastName,nationality,birthCity,primaryPosition,birthDate,birthStateProvince,height,height_cm,weight,shootsCatches
0,8466148,Marian,Hossa,SVK,Stará Lubovna,RW,1979-01-12 00:00:00,NaN,"6' 1""",185.42,207.0,L
1,8465058,Michal,Rozsival,CZE,Vlasim,D,1978-09-03 01:00:00,NaN,"6' 1""",185.42,210.0,R
2,8476906,Shayne,Gostisbehere,USA,Pembroke Pines,D,1993-04-20 01:00:00,FL,"5' 11""",180.34,180.0,L
3,8466285,Brian,Campbell,CAN,Strathroy,D,1979-05-23 01:00:00,ON,"5' 10""",177.80,192.0,L
4,8470607,Brent,Seabrook,CAN,Richmond,D,1985-04-20 01:00:00,BC,"6' 3""",190.50,220.0,R


In [9]:
#store team info csv into dataframe
team_info_df = pd.read_csv('data/team_info.csv')
team_info_df.head()

,team_id,franchiseId,shortName,teamName,abbreviation,link
0,1,23,New Jersey,Devils,NJD,/api/v1/teams/1
1,4,16,Philadelphia,Flyers,PHI,/api/v1/teams/4
2,26,14,Los Angeles,Kings,LAK,/api/v1/teams/26
3,14,31,Tampa Bay,Lightning,TBL,/api/v1/teams/14
4,6,6,Boston,Bruins,BOS,/api/v1/teams/6


In [10]:
#create new dataframe with select columms
new_player_info_df = player_info_df [['player_id', 'firstName', 'lastName', 'nationality']].copy()
new_player_info_df .head()

,player_id,firstName,lastName,nationality
0,8466148,Marian,Hossa,SVK
1,8465058,Michal,Rozsival,CZE
2,8476906,Shayne,Gostisbehere,USA
3,8466285,Brian,Campbell,CAN
4,8470607,Brent,Seabrook,CAN


Merging dataframes to create a single data set to be used to train machine learning AI

In [11]:
#merging cleaned player info + game skater stats
merge_1_df = new_player_info_df.merge(new_game_skater_df, how='left', left_on='player_id', right_on='player_id')
merge_1_df.head()

,player_id,firstName,lastName,nationality,timeOnIce,assists,goals,shots,blocked,powerPlayGoals,powerPlayAssists,shortHandedGoals,shortHandedAssists
0,8466148,Marian,Hossa,SVK,723.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0
1,8466148,Marian,Hossa,SVK,1011.0,0.0,1.0,4.0,0.0,1.0,0.0,0.0,0.0
2,8466148,Marian,Hossa,SVK,727.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
3,8466148,Marian,Hossa,SVK,993.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
4,8466148,Marian,Hossa,SVK,1110.0,1.0,1.0,3.0,0.0,0.0,0.0,0.0,1.0


In [14]:
#merging cleaned goalie info + team info
merge_2_df = team_info_df.merge(goalie_stats_df, how='right', left_on='team_id', right_on='team_id')
merge_2_df


,team_id,franchiseId,shortName,teamName,abbreviation,link,game_id,player_id,timeOnIce,assists,...,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decision,savePercentage,powerPlaySavePercentage,evenStrengthSavePercentage
0,1,23.0,New Jersey,Devils,NJD,/api/v1/teams/1,2016020107,8476234,3664,0,...,6,0,20,0,21,8,L,89.655172,75.0,95.238095
1,1,23.0,New Jersey,Devils,NJD,/api/v1/teams/1,2015020981,8476234,3600,0,...,4,3,17,3,22,5,L,80.000000,80.0,77.272727
2,1,23.0,New Jersey,Devils,NJD,/api/v1/teams/1,2017020482,8471239,3690,0,...,0,0,34,0,36,0,L,94.444444,NaN,94.444444
3,1,23.0,New Jersey,Devils,NJD,/api/v1/teams/1,2015020453,8471239,3600,0,...,3,0,22,0,22,3,W,100.000000,100.0,100.000000
4,1,23.0,New Jersey,Devils,NJD,/api/v1/teams/1,2017020952,8476234,3600,0,...,2,1,27,1,28,2,W,96.774194,100.0,96.428571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56651,89,NaN,NaN,NaN,NaN,NaN,2019040652,8476945,600,0,...,0,0,10,0,16,0,NaN,62.500000,NaN,62.500000
56652,89,NaN,NaN,NaN,NaN,NaN,2018040643,8471469,583,0,...,0,0,6,0,10,0,NaN,60.000000,NaN,60.000000
56653,89,NaN,NaN,NaN,NaN,NaN,2018040643,8471227,600,0,...,0,0,6,0,11,0,NaN,54.545455,NaN,54.545455
56654,89,NaN,NaN,NaN,NaN,NaN,2018040643,8471469,583,0,...,0,0,6,0,10,0,NaN,60.000000,NaN,60.000000


In [15]:
training_data_df = merge_2_df.merge(merge_1_df, how='left', left_on='player_id', right_on='player_id')
training_data_df

,team_id,franchiseId,shortName,teamName,abbreviation,link,game_id,player_id,timeOnIce_x,assists_x,...,nationality,timeOnIce_y,assists_y,goals_y,shots_y,blocked,powerPlayGoals,powerPlayAssists,shortHandedGoals,shortHandedAssists
0,1,23.0,New Jersey,Devils,NJD,/api/v1/teams/1,2016020107,8476234,3664,0,...,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,23.0,New Jersey,Devils,NJD,/api/v1/teams/1,2015020981,8476234,3600,0,...,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,23.0,New Jersey,Devils,NJD,/api/v1/teams/1,2017020482,8471239,3690,0,...,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,23.0,New Jersey,Devils,NJD,/api/v1/teams/1,2015020453,8471239,3600,0,...,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,23.0,New Jersey,Devils,NJD,/api/v1/teams/1,2017020952,8476234,3600,0,...,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56651,89,NaN,NaN,NaN,NaN,NaN,2019040652,8476945,600,0,...,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56652,89,NaN,NaN,NaN,NaN,NaN,2018040643,8471469,583,0,...,FIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56653,89,NaN,NaN,NaN,NaN,NaN,2018040643,8471227,600,0,...,CAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56654,89,NaN,NaN,NaN,NaN,NaN,2018040643,8471469,583,0,...,FIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
training_data_df.to_csv("training_data.csv")

In [21]:
#creat fantasy points dataframe for skaters
fantasy_skater_data = {'Stats': ['Assists', 'Blocked Shots', 'Goals','Power Play Assists', 'Power Play Goals','Shots on Goal', 'Short Handed Assists', 'Short Handed Goals'], 
                'Points': [8, 1.6, 12, 0.5, 0.5, 1.6, 2, 2 ]}
fantasy_skater_df = pd.DataFrame(fantasy_skater_data)
fantasy_skater_df

,Stats,Points
0,Assists,8.0
1,Blocked Shots,1.6
2,Goals,12.0
3,Power Play Assists,0.5
4,Power Play Goals,0.5
5,Shots on Goal,1.6
6,Short Handed Assists,2.0
7,Short Handed Goals,2.0


In [22]:
#create fantasy points datafram for goalies
fantasy_goalie_data = {'Stats': ['Goals Against', 'Shutouts', 'Saves', 'Wins'],
                      'Points': [-4, 8, 0.8, 12]}
fantasy_goalie_df = pd.DataFrame(fantasy_goalie_data)
fantasy_goalie_df

,Stats,Points
0,Goals Against,-4.0
1,Shutouts,8.0
2,Saves,0.8
3,Wins,12.0
